<a href="https://colab.research.google.com/github/SURESHBEEKHANI/Autoencoders/blob/main/convolutional_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x

In [ ]:
%pylab inline

In [ ]:
#import the tensorflow import the keras
from tensorflow import keras

In [ ]:
#load the dataset
(x_train, _),(x_test, _)=keras.datasets.mnist.load_data()

In [ ]:
#normilize the data range into [0,1]
x_train=x_train/255.0
x_test=x_test/255.0

In [ ]:

# Define the encoder model using convolutional layers
encoder = keras.models.Sequential([
    # Reshape the input into a 28x28x1 image (grayscale)
    keras.layers.Reshape([28, 28, 1], input_shape=[28, 28]),
    # First convolutional layer with 16 filters, each of size 3x3, ReLU activation, and padding to maintain size
    keras.layers.Conv2D(16, kernel_size=(3, 3), padding="same", activation="relu"),
    # Max pooling layer with pool size 2x2 to downsample the spatial dimensions
    keras.layers.MaxPool2D(pool_size=2),
    # Second convolutional layer with 32 filters, each of size 3x3, ReLU activation, and padding
    keras.layers.Conv2D(32, kernel_size=(3, 3), padding="same", activation="relu"),
    # Max pooling layer
    keras.layers.MaxPool2D(pool_size=2),
    # Third convolutional layer with 64 filters, each of size 3x3, ReLU activation, and padding
    keras.layers.Conv2D(64, kernel_size=(3, 3), padding="same", activation="relu"),
    # Max pooling layer
    keras.layers.MaxPool2D(pool_size=2)
])

# Print the model summary
encoder.summary()

In [ ]:
# Predict the encoded representation of a single test image and print its shape
encoder.predict(x_test[0].reshape((1, 28, 28))).shape


In [ ]:

# Define the decoder model using convolutional transpose layers
decoder = keras.models.Sequential([
    # Convolutional transpose layer with 32 filters, each of size 3x3, stride 2, ReLU activation, and valid padding
    keras.layers.Conv2DTranspose(32, kernel_size=(3, 3), strides=2, padding="valid",
                                 activation="relu",
                                 input_shape=[3, 3, 64]),
    # Convolutional transpose layer with 16 filters, each of size 3x3, stride 2, ReLU activation, and same padding
    keras.layers.Conv2DTranspose(16, kernel_size=(3, 3), strides=2, padding="same",
                                 activation="relu"),
    # Convolutional transpose layer with 1 filter, size 3x3, stride 2, sigmoid activation, and same padding
    keras.layers.Conv2DTranspose(1, kernel_size=(3, 3), strides=2, padding="same",
                                 activation="sigmoid"),
    # Reshape the output into the original image shape (28x28)
    keras.layers.Reshape([28, 28])
])

# Print the model summary
decoder.summary()

In [ ]:
#Combine the both encoder two decoder
convolutional_autoencoder = keras.models.Sequential([encoder, decoder])

# Print he model summary
convolutional_autoencoder.summary()

In [ ]:
#Complie the convolutional_autoencoder
convolutional_autoencoder.compile(loss="binary_crossentropy",
                                  optimizer='adam'
)

In [ ]:
#Train the model
convolutional_autoencoder.fit(x_train, x_train, epochs=10,validation_data=(x_test, x_test))

In [ ]:
# Set the figure size for the plot
plt.figure(figsize=(20, 5))
plt.title("""Original vs. Reconstructed""")


# Iterate over 8 examples from the test set
for i in range(4):
    # Plot the original image from the test set
    plt.subplot(2, 8, i + 1)
    plt.title("Original")
    plt.imshow(x_test[i], cmap="binary")


    # Make a prediction using the convolutional autoencoder on the current test image
    pred = convolutional_autoencoder.predict(x_test[i].reshape((1, 28, 28, 1)))

    # Plot the reconstructed image by the convolutional autoencoder
    plt.subplot(2, 8, i + 8 + 1)
    plt.title("Reconstructed")
    plt.imshow(pred.reshape((28, 28)), cmap="binary")


plt.tight_layout()
plt.show()

In [ ]:
# Set the figure size for the plot
figsize(15, 15)

# Iterate over all filters in the last convolutional layer of the encoder
for i in range(8 * 8):
    # Plot each filter as a subplot in an 8x8 grid
    subplot(8, 8, i+1)
    # Display the weights (filters) of the convolutional layer
    imshow(encoder.layers[-2].weights[0][:, :, 0, i])
